**Meta Learning**: in this notebook, we tested meta learning on our dataset. We will interpret the results at the end, but in short we did not actually obtain better results, or noticeable improvement at least.

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
import random as rd
import pickle 
from sklearn.feature_selection import SelectKBest, chi2 ,f_classif
!pip install learn2learn
import learn2learn as l2l
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.model_selection import train_test_split, learning_curve, ShuffleSplit, GridSearchCV
from sklearn.metrics import accuracy_score 
from torch import nn, optim
from sklearn.metrics import f1_score, log_loss, roc_auc_score
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable as V
import random

In [ ]:
def get_selected_features_df(X_df, f_i):
  X_selected = X_df.copy()
  i = 0
  for col in X_df.iloc[:, 4:]:
    if i not in f_i: 
      X_selected.pop(col)
    i += 1
  return X_selected

fi = []
train_data = []
with open('/content/drive/My Drive/Colab Notebooks/features_indexes.data', 'rb') as f:
  fi = pickle.load(f) 

# train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/X_afterSteps_12.csv') 
train_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/train_features.csv') 
df_y = pd.read_csv("/content/drive/My Drive/Colab Notebooks/train_targets_scored.csv")
train_labels = np.array(df_y.iloc[:, 1:])

train_data = get_selected_features_df(train_data, fi)

x = train_data.iloc[:, 2:]

def encode(df, col):
  new_df = df.copy()
  classes = list(pd.unique(new_df.iloc[:,col]))
  for i in range(len(df.iloc[:,col])):
    new_df.iloc[i, col] = classes.index(new_df.iloc[i, col])
  return new_df

x=encode(x,0)
x=encode(x,1) 
x=encode(x,2)
print(x)
y = df_y.iloc[:, 1:]
X = x

# split data and convert to tensors
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
X_train = np.array(X_train)
y_train = np.array(y_train)
X_test = np.array(X_test)
y_test = np.array(y_test)

temp = []
for i in X_train:
    temp.append(i) 
X_train_tensor = torch.Tensor(temp)

temp = []
for i in X_test:
  temp.append(i)
X_test_tensor = torch.Tensor(temp)
y_train_tensor = torch.Tensor(y_train)
y_test_tensor = torch.Tensor(y_test)

       cp_time cp_dose      g-0     g-7  ...    c-96    c-97    c-98    c-99
0            0       0      0.0 -0.0326  ... -0.3981  0.2139  0.3801  0.4176
1            1       0      1.0  0.3372  ...  0.1522  0.1241  0.6077  0.7371
2            2       0      2.0  0.2155  ... -0.6417 -0.2187 -1.4080  0.6931
3            2       0      3.0  0.1792  ... -1.6210 -0.8784 -0.3876 -0.8154
4            1       1      4.0 -0.1498  ...  0.1094  0.2885 -0.3786  0.7125
...        ...     ...      ...     ...  ...     ...     ...     ...     ...
23809        0       1    527.0  0.3055  ...  0.0631  0.9171  0.5258  0.4680
23810        0       1   1138.0 -0.5565  ... -0.2084 -0.1224 -0.2715  0.3689
23811        2       1  14364.0  0.1745  ...  0.2256  0.7592  0.6656  0.3808
23812        0       0  14365.0  0.0463  ...  0.1732  0.7015 -0.6290  0.0740
23813        1       0  14366.0  0.9146  ... -3.5770 -0.4775 -2.1500 -4.2520

[23814 rows x 395 columns]


In [ ]:
# Deep NN Model we defined for the sake of this example 
class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(X.shape[1], 256)  
        self.fc2 = nn.Linear(256, 1024) 
        self.fc3 = nn.Linear(1024, 512) 
        self.fc4 = nn.Linear(512, 206) 

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(nn.Dropout(p=0.1)(self.fc2(x)))
        x = F.relu(nn.Dropout(p=0.1)(self.fc3(x)))
        x = self.fc4(x)
        return torch.sigmoid(x)

net = Net()
print(net)

Net(
  (fc1): Linear(in_features=395, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=1024, bias=True)
  (fc3): Linear(in_features=1024, out_features=512, bias=True)
  (fc4): Linear(in_features=512, out_features=206, bias=True)
)


In [ ]:
X_train = X_train_tensor
y_train = y_train_tensor
X_test = X_test_tensor
y_test = y_test_tensor

def compute_loss(task_model):
  target = y_train_tensor 
  output = task_model(X_train_tensor) 
  return nn.BCELoss()(output, target)

model = net
maml = l2l.algorithms.MAML(model, lr=0.001)
opt = torch.optim.SGD(maml.parameters(), lr=0.001)
for iteration in range(100): 
    opt.zero_grad() 
    task_model = maml.clone()  
    adaptation_loss = compute_loss(task_model)
    task_model.adapt(adaptation_loss)  # computes updated gradient, update task_model
    evaluation_loss = compute_loss(task_model)
    evaluation_loss.backward()  # gradients w.r.t. maml.parameters()
    opt.step()  

In [ ]:
outputs = model(X_train_tensor)
outputs[outputs >= 0.5] = 1 
outputs[outputs < 0.5] = 0 
accuracy = (outputs == y_train_tensor).sum() / (y_train_tensor.shape[0]*y_train_tensor.shape[1]) * 100
print("Accuracy: " + str(accuracy) + "%")  

f1 = f1_score(y_train_tensor.detach().numpy(), outputs.detach().numpy(), average='weighted')
print("f1-score: " + str(f1)) 

Accuracy: tensor(53.7608)%
f1-score: 0.014947048045235035


As we can see, meta learning applied to our dataset did not actually show better results than usual neural network we tried earlier. The results are similar and both of them are not good results. Also note that with and without feature selection we could not obtain better results. We will do the hypothesis testing to experiment more on the differences between these models even though all of them had bad results as in accuracy and f1-score which is mainly what we care about. NOTE that one model we tried predicted all 0's and had accuracy 99% because of its false positives and f1-score of almost 0, it is still a bad model.
I belive our problem is more complicated than just getting help from different techniques such as meta learning. It needs careful data analysis and preprocessing, which requires an expert in the field since some cells data cannot be messed with even if it's for the purpose of machine learning. 
Some techniques that might help is the one we applied in another section of this phase, which is close to oversampling 